In [ ]:
%matplotlib agg
from river.datasets import synth
from clustream import CluStream
from river import drift
import math
import numpy as np
import matplotlib.pyplot as plt
from river.utils import dict2numpy
from scipy.spatial import distance_matrix
from time import time
from river import stream
import pandas as pd

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score



In [ ]:
ds = pd.read_csv('random.csv')
ds.dropna(axis=0,inplace=True)
y = ds.pop('.class')

learner = CluStream(n_macro_clusters=3,max_micro_clusters=52,time_window=200,micro_cluster_r_factor=0.01)



In [ ]:
fig, ax = plt.subplots(nrows=2,ncols=2,figsize = (10,10))
lst1 = []
lst2 = []
lst3 = []
i = 1
k,m = 0,0
#tup = [(0,0),(0,1),(0,2),(1,0),(1,1),(1,2),(2,0),(2,1),(2,2)]
tup = [(0,0),(0,1),(1,0),(1,1)]
l = 0
for x,y in stream.iter_pandas(ds,y):
    
    learner = learner.learn_one(x)
    pred  = learner.predict_one(x)
    
    if y == 1.0 :
        lst1.append([x['X1'],x['X2']])
    elif y == 2.0 :
        lst2.append([x['X1'],x['X2']])
    else :
        lst3.append([x['X1'],x['X2']])

    if i > 2000 :
        break
   
    if i % 200 == 0 and i <= 800 :
        k,m = tup[l]
        centers = learner.get_micro_clusters()

        learner.offline_cluster()
        centroids = learner.centers

        data = np.concatenate([lst1,lst2,lst3],axis= 0)

        print(silhouette_score(data,learner.offline_predict(data)))
    

        lst1 = np.array(lst1)
        lst2 = np.array(lst2) 
        lst3 = np.array(lst3) 
        ax[k,m].scatter(lst1[:,0],lst1[:,1])
        ax[k,m].scatter(lst2[:,0],lst2[:,1])
        ax[k,m].scatter(lst3[:,0],lst3[:,1])
        ax[k,m].scatter(centers[:,0],centers[:,1])
        ax[k,m].scatter(centroids[:,0],centroids[:,1],marker="x",c= 'black')


      
        ax[k,m].set_title(f'T = {i}')
        l += 1
        lst1 = []
        lst2 = []
        lst3 = []
            


    i += 1 


fig.savefig('concept_drift.jpg')